In [1]:
import pandas as pd
import urllib
import feedparser
import atoma
import time
import requests

from pathlib import Path

from collections import defaultdict

In [2]:
base_url = 'http://export.arxiv.org/api/query?'

In [213]:
search_query_story = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+%28abs:narrative+OR+abs:story+OR+abs:fiction+OR+abs:plot%29'

search_query_tables = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+%28abs:tables+OR+abs:data+OR+abs:structured+OR+abs:table-to-text%29'

search_query_games = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+abs:games'

search_query_knowledge = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+%28abs:knowledge+OR+abs:graphs+OR+abs:semantics%29'

search_query_poetry = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+%28abs:poetry+OR+abs:poems+OR+abs:lyrics%29'

search_query_dialogue = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+%28abs:dialogue+OR+abs:agents+OR+abs:conversation%29'

search_query_images = '%28abs:"text%20generation"+OR+abs:"natural%20language%20generation"%29+AND+%28abs:images+OR+abs:image2text+OR+abs:description+OR+abs:image-to-text+OR+abs:caption%29'

In [214]:
queries = {"story": search_query_story, 
           "tables": search_query_tables, 
           "games": search_query_games, 
           "dialog": search_query_dialogue, 
           "knowledge": search_query_knowledge, 
           "poetry": search_query_poetry, 
           "image2text": search_query_images}

sort = "&sortBy=lastUpdatedDate&sortOrder=descending"

start = 0                     # retreive the first 5 results
max_results = 200
#query = 'search_query=%s&start=%i&max_results=%i' % (query,
                                                    # start,
                                                     #max_results) + sort

In [132]:

urllib.parse.quote(base_url + query)

'http%3A//export.arxiv.org/api/query%3Fsearch_query%3Dabs%3A%22text%2520generation%22AND%2528abs%3AdialogueORabs%3Aagents%2529%26start%3D0%26max_results%3D50%26sortBy%3DlastUpdatedDate%26sortOrder%3Ddescending'

In [94]:
#feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
#feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

# perform a GET request using the base_url and query
response = urllib.request.urlopen(base_url+query).read()

# parse the response using feedparser
feed = feedparser.parse(response)


In [95]:
feed.keys()

dict_keys(['bozo', 'entries', 'feed', 'headers', 'encoding', 'version', 'namespaces'])

In [96]:
feed

{'bozo': False,
 'entries': [{'id': 'http://arxiv.org/abs/2101.02690v1',
   'guidislink': True,
   'link': 'http://arxiv.org/abs/2101.02690v1',
   'updated': '2021-01-07T18:52:08Z',
   'updated_parsed': time.struct_time(tm_year=2021, tm_mon=1, tm_mday=7, tm_hour=18, tm_min=52, tm_sec=8, tm_wday=3, tm_yday=7, tm_isdst=0),
   'published': '2021-01-07T18:52:08Z',
   'published_parsed': time.struct_time(tm_year=2021, tm_mon=1, tm_mday=7, tm_hour=18, tm_min=52, tm_sec=8, tm_wday=3, tm_yday=7, tm_isdst=0),
   'title': 'Theorem Proving and Algebra',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': '',
    'value': 'Theorem Proving and Algebra'},
   'summary': 'This book can be seen either as a text on theorem proving that uses\ntechniques from general algebra, or else as a text on general algebra\nillustrated and made concrete by practical exercises in theorem proving. The\nbook considers several different logical systems, including first-order logic,\nHorn clause l

In [ ]:
https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=text+generation&terms-0-field=abstract&terms-1-operator=AND&terms-1-term=narrative+generation&terms-1-field=abstract&terms-2-operator=OR&terms-2-term=story&terms-2-field=title&classification-computer_science=y&classification-physics_archives=all&classification-include_cross_list=include&date-filter_by=all_dates&date-year=&date-from_date=&date-to_date=&date-date_type=submitted_date_first&abstracts=hide&size=50&order=-announced_date_first

In [155]:
response = requests.get(base_url + query)
feed = atoma.parse_atom_bytes(response.content)

In [215]:
from collections import defaultdict

arts2 = {}

cats_of_interest = ['cs.CL', 'cs.HC', 'cs.AI', 'cs.DL', 'cs.GT', 'cs.GL', 'cs.IR', 'cs.DB',
                               'cs.IT', 'cs.LG', 'cs.MA', 'cs.MM', 'cs.NE', 'cs.SI']
queries = {"story": search_query_story, 
           "table2text": search_query_tables, 
           "games": search_query_games, 
           "dialogue": search_query_dialogue, 
           "knowledge": search_query_knowledge, 
           "poetry": search_query_poetry, 
           "image2text": search_query_images}

sort = "&sortBy=lastUpdatedDate&sortOrder=descending"
start = 0                     # retreive the first 150 results
max_results = 200

for searchtype, querystring in queries.items():
    query = 'search_query=%s&start=%i&max_results=%i' % (querystring,
                                                     start,
                                                     max_results) + sort
    response = requests.get(base_url + query)
    feed = atoma.parse_atom_bytes(response.content)
    arts2[searchtype] = []
    for entry in feed.entries:
        cats = []
        for cat in entry.categories:
            cats.append(cat.term)
        if not set(cats).intersection(set(cats_of_interest)):
            continue
        pubdate = entry.published
        title = entry.title.value
        authors = []
        for author in entry.authors:
            authors.append(author.name)
        abslink = entry.id_
        text = entry.summary.value
        data = {'title': title, 'pubdate': pubdate, 'id': abslink, 
                'authors': ', '.join(authors), 'categories': ', '.join(cats),
               'search': searchtype, 'abstract': text}
        arts2[searchtype].append(data)
        time.sleep(2)

In [6]:
arts2.keys()

dict_keys(['story', 'table2text', 'games', 'dialogue', 'knowledge', 'poetry', 'image2text'])

In [180]:
arts3 = dict(zip(queries.keys(), arts2.values()))

In [181]:
arts3.keys()

dict_keys(['story', 'tables', 'games', 'dialogue', 'knowledge', 'poetry', 'image2text'])

In [219]:
for key, val in arts2.items():
    print(key, len(val))

story 33
table2text 194
games 13
dialogue 1
knowledge 194
poetry 12
image2text 108


In [188]:
import more_itertools

In [307]:
for x in more_itertools.chunked(arts2['knowledge'], 100):
    print(len(x))

100
33


In [19]:
# write the csv data files

import numpy as np

def create_df_from_new_vals(vals):
    # scraped new data
    df = pd.DataFrame(vals)
    df['pubdate'] = pd.to_datetime(df['pubdate'])
    df.sort_values(by=["pubdate", "title"], ascending=False)
    df['displaydate'] = df['pubdate'].dt.strftime('%Y-%m-%d')
    return df

def read_data_csv(filename):
    # read old data files
    df = pd.read_csv(filename)
    return df

def read_data_csv_date(date, categ):
    path = Path(f"_data/{categ}/").glob(f"{date}-{categ}-*.csv")
    file = [str(x) for x in path][0]
    df = pd.read_csv(file)
    df.sort_values(by="pubdate", ascending=False)
    return df

def write_data_file(vals, key, tag=None, columns=['title','pubdate','id','authors','categories','search','abstract','displaydate']):
    # writes a dataframe with today's date or top date - assumes already created cols
    df = pd.DataFrame(vals, columns=columns)
    df.sort_values(by="pubdate", inplace=True, ascending=False)  # fix sort order for display
    date = df.iloc[0]['displaydate']  # if it's a full list, use the top of the list as date
    print("writing new file", date)
    if tag:
        df.to_csv(f'_data/{key}/{tag}-' + date + '-' + key + '-' + str(len(df)) + '.csv', index=None)
    else:
        df.to_csv(f'_data/{key}/' + date + '-' + key + '-' + str(len(df)) + '.csv', index=None)
    return date

def merge_with_previous(category, newdata):
    most_recent = get_latest_datafile(category)
    date, cat, count = parse_filename(most_recent)
    newdf = create_df_from_new_vals(newdata)
    df = pd.read_csv(most_recent)
    if int(count) < 100:
        newdf = pd.concat([newdf, df]).drop_duplicates(subset="id")  # should we then delete the last most recent?
        newdf.sort_values(by="pubdate", inplace=True, ascending=False)
    # else what? also what about len(new)
    if len(df) == len(newdf):
        return [], None
    else:
        return newdf, date  # date of most recent, to delete
    
def write_new_data(categ, newdata):
    # newdata is list of values not yet a df
    # use the crawled new data and join it with old data files
    newdf, prevdate = merge_with_previous(categ, newdata)  # newdf is sorted
    # use date to remove the .md file of that date
    newfiledates = []
    if len(newdf):
        newdf.sort_values(by='pubdate', ascending=True)
        newdfdata = newdf.values.tolist()
        columns = newdf.columns
        if len(newdata) > 120:
            for x in more_itertools.chunked(newdfdata, 60):
                new = write_data_file(x, categ, columns=columns, tag=None)
                newfiledates.append(new)
        else:
            new = write_data_file(newdfdata, categ, columns=columns, tag=None)
            newfiledates.append(new)
    else:
        print("no new data to write")
    return newfiledates, prevdate

def write_all_data_files(categ, df, tag=None):
    # just for cleanup, dont use in prod
    columns = df.columns
    df.sort_values(by="pubdate", ascending=True, inplace=True)  # you want the old ones written first as 100's
    if len(df) > 100:
        for x in more_itertools.chunked(df.values.tolist(), 100):
            write_data_file(x, categ, tag=tag, columns=columns)
    else:
        write_data_file(df.values.tolist(), categ, tag=tag, columns=columns)

In [19]:
data = arts2['story']

In [20]:
data

[{'title': 'Political Depolarization of News Articles Using Attribute-aware Word\n  Embeddings',
  'pubdate': datetime.datetime(2021, 1, 5, 7, 39, 12, tzinfo=tzutc()),
  'id': 'http://arxiv.org/abs/2101.01391v1',
  'authors': 'Ruibo Liu, Lili Wang, Chenyan Jia, Soroush Vosoughi',
  'categories': 'cs.CL, cs.AI',
  'search': 'story',
  'abstract': 'Political polarization in the US is on the rise. This polarization negatively\naffects the public sphere by contributing to the creation of ideological echo\nchambers. In this paper, we focus on addressing one of the factors that\ncontributes to this polarity, polarized media. We introduce a framework for\ndepolarizing news articles. Given an article on a certain topic with a\nparticular ideological slant (eg., liberal or conservative), the framework\nfirst detects polar language in the article and then generates a new article\nwith the polar language replaced with neutral expressions. To detect polar\nwords, we train a multi-attribute-aware w

In [463]:
for key, vals in arts2.items():
    write_data(key, vals)

2021-01-12 story 23
no new data to write
2021-01-12 table2text 44
writing new file 2021-01-13
2021-01-12 games 10
no new data to write
2021-01-12 dialogue 30
no new data to write
2021-01-12 knowledge 33
writing new file 2021-01-13
2021-01-12 poetry 10
no new data to write
2021-01-12 image2text 74
no new data to write


In [417]:
merge_with_previous("table2text", None)

['2021-01-05-table2text-100', '2021-01-12-table2text-44']
2021-01-12 table2text 44
('2021-01-12', 'table2text', '44')


In [207]:
sorted(['2020-01-30-story', '2019-08-23-dialogue', '2019-04-11','2019-08-20-story', '2020-08-20-dialogue', '2020-11-19-abs'])

['2019-04-11',
 '2019-08-20-story',
 '2019-08-23-dialogue',
 '2020-01-30-story',
 '2020-08-20-dialogue',
 '2020-11-19-abs']

In [310]:
# read in a file



In [268]:
data = read_csv("_data/2020-04-21-tables-44.csv")

In [269]:
data

,title,pubdate,id,authors,categories,search,displaydate
0,Learning to Encode Evolutionary Knowledge for ...,2020-04-21 13:09:50+00:00,http://arxiv.org/abs/2004.09974v1,"Canxiang Yan, Jianhao Yan, Yangyin Xu, Cheng N...","cs.CL, cs.LG",tables,2020-04-21
1,Neural Data-to-Text Generation with Dynamic Co...,2020-04-16 02:50:51+00:00,http://arxiv.org/abs/2004.07426v2,"Kai Chen, Fayuan Li, Baotian Hu, Weihua Peng, ...",cs.CL,tables,2020-04-16
2,Machine Translation Pre-training for Data-to-T...,2020-04-05 02:47:16+00:00,http://arxiv.org/abs/2004.02077v1,"Mihir Kale, Scott Roy",cs.CL,tables,2020-04-05
3,CG-BERT: Conditional Text Generation with BERT...,2020-04-04 07:31:59+00:00,http://arxiv.org/abs/2004.01881v1,"Congying Xia, Chenwei Zhang, Hoang Nguyen, Jia...","cs.CL, cs.LG",tables,2020-04-04
4,"Heavy-tailed Representations, Text Polarity Cl...",2020-03-25 19:24:05+00:00,http://arxiv.org/abs/2003.11593v1,"Hamid Jalalzai, Pierre Colombo, Chloé Clavel, ...","stat.ML, cs.CL, cs.LG",tables,2020-03-25
5,Unsupervised Pidgin Text Generation By Pivotin...,2020-03-18 15:27:35+00:00,http://arxiv.org/abs/2003.08272v1,"Ernie Chang, David Ifeoluwa Adelani, Xiaoyu Sh...",cs.CL,tables,2020-03-18
6,Generating Major Types of Chinese Classical Po...,2020-03-13 14:16:25+00:00,http://arxiv.org/abs/2003.11528v1,"Jinyi Hu, Maosong Sun",cs.CL,tables,2020-03-13
7,Meta-CoTGAN: A Meta Cooperative Training Parad...,2020-03-12 04:47:52+00:00,http://arxiv.org/abs/2003.11530v1,"Haiyan Yin, Dingcheng Li, Xu Li, Ping Li","cs.CL, cs.LG, stat.ML",tables,2020-03-12
8,Generating Natural Language Adversarial Exampl...,2020-03-10 03:21:35+00:00,http://arxiv.org/abs/2003.10388v1,"Yankun Ren, Jianbin Lin, Siliang Tang, Jun Zho...","cs.CL, cs.LG, stat.ML",tables,2020-03-10
9,Plug and Play Language Models: A Simple Approa...,2019-12-04 18:32:15+00:00,http://arxiv.org/abs/1912.02164v4,"Sumanth Dathathri, Andrea Madotto, Janice Lan,...","cs.CL, cs.AI, cs.LG",tables,2019-12-04


In [274]:
df.columns

Index(['title', 'id', 'authors', 'categories'], dtype='object')

In [54]:
def make_date_path(date):
    return date.replace('-', '/') + '/'

In [216]:
# where the page ends up published by category - add to a data file?
def make_entry_for_md(date, categ):
    filepath = 'categories/' + categ + '/' + make_date_path(date) + categ + '.html'
    return filepath

In [205]:
def write_table_in_md(df, handle):
    df = df[['title', 'authors', 'categories', 'id', 'displaydate']]
    headers = list(df.columns)
    headers.remove('id')
    handle.write("\n")
    handle.write(f"### Written on {today}\n\n")
    handle.write('| ' + ' | '.join(headers) + " |\n")
    handle.write('| ' + ' | '.join(['-----' for x in range(len(headers))]) + ' |\n')
    for i, row in df.iterrows():
        title = row['title'].replace('\n','')
        titlelink = f"[{title}]({row['id']})"
        authors = row['authors']
        categories = row['categories']
        ddate = row['displaydate']
        items = [titlelink, authors, categories, ddate]
        handle.write('| ' + ' | '.join(items) + ' |\n')
    handle.write('\n')
    return

In [218]:
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')

def write_table_md(df, date, categ, prevlink, most_recent=False):
    today = datetime.today().strftime('%Y-%m-%d')
    with open('categories/' + categ + '/_posts/' + date + '-' + categ + '.md', 'w') as handle:
        handle.write("---\n")
        handle.write('category: ' + categ + '\n')
        handle.write('layout: post\n')
        handle.write('sidebar:\n')
        handle.write('  nav: contents\n')
        handle.write('---\n\n')
        write_table_in_md(df, handle)
        if type(prevlink) == str:
            handle.write(f'[Previous]({prevlink})\n')
    if most_recent:
        # write the main page too
        with open('categories/' + categ + '/' + categ + '.md', 'w') as handle:
            handle.write("---\n")
            handle.write('category: ' + categ + '\n')
            handle.write('layout: page\n')
            handle.write('title:' + categ + '\n')
            handle.write('sidebar:\n')
            handle.write('  nav: contents\n')
            handle.write('---\n\n')
            write_table_in_md(df, handle)
            if type(prevlink) == str:
                handle.write(f'[Previous]({prevlink})\n')
    print("wrote ", 'categories/' + categ + '/_posts/' + date + '-' + categ + '.md')

In [17]:
datafiles = !ls _data/*.csv

In [298]:
write_table_md(data, 'table2text', today)

In [18]:
datafiles

['ls: _data/*.csv: No such file or directory']

In [320]:
from pathlib import Path

In [116]:
def delete_prev_files(date, category):
    print("would here delete", date, category)
    rem_file = Path(f"categories/{category}/_posts/{date}-{category}.md")
    if rem_file.is_file():
        #rem_file.unlink()
        print('would delete', rem_file)
    rem_data_files = Path(f"_data/{category}/").glob(f"{date}-{category}*.csv")
    rem_data_files = [str(x) for x in rem_data_files]
    if rem_data_files[0]:
        #Path(rem_data_file[0]).unlink()
        print('would delete', rem_data_files[0])
    (year, month, day) = date.split('-')
    rem_html_file = Path(f"_site/{category}/{year}/{month}/{day}/{category}.html")
    if rem_html_file.is_file():
        #rem_data_file[0].unlink()
        print('would delete', rem_html_file)
    return

In [470]:
def check_for_filedate(date, category):
    path = Path(f"categories/{category}/_posts/{date}-{category}.md")
    if path.is_file():
        return True
    else:
        return False

In [51]:
def parse_filename(filename):
    path = Path(filename)
    parts = path.stem.split('-')
    date = '-'.join(parts[0:3])
    cat = parts[3]
    try:
        count = parts[4]
    except:
        count = None
    print(date, cat, count)
    return (date, cat, count)

In [486]:
categs = arts2.keys()
categs

dict_keys(['story', 'table2text', 'games', 'dialogue', 'knowledge', 'poetry', 'image2text'])

In [27]:
def get_latest_datafile(categ):
    files = Path(f'_data/{categ}/').glob("*" + categ + "*.csv")
    files = sorted([str(file) for file in files])
    print("all files for categ", files)
    print("most recent", files[-1])
    return files[-1]

In [28]:
get_latest_datafile("story")

all files for categ ['_data/story/2021-01-12-story-23.csv']
most recent _data/story/2021-01-12-story-23.csv


'_data/story/2021-01-12-story-23.csv'

In [61]:
newdata = get_new_data('story')

23


In [76]:
def get_written_files_categ(categ):
    df = pd.read_csv("_data/files_written.csv")
    categs = df.groupby('category')
    return categs.get_group(categ).sort_values(by="date", ascending=False)

In [103]:
def remove_prevdate_from_record(prevdate, categ):
    # also remove prev data file?  fix new prevlinks
    df = pd.read_csv("_data/files_written.csv")
    df.set_index('md_filename', inplace=True)
    print("would delete here:", df.loc[f"{prevdate}-{categ}.md"], len(df))
    #del df.loc[f"{prevdate}-{categ}.md"]
    print(len(df))
    df.to_csv("_data/files_written.csv")

In [130]:
def add_to_record_get_prevlink(newdate, categ):
    df = pd.read_csv("_data/files_written.csv")
    subset = df[df['category'] == categ]
    subset.sort_values(by="date", ascending=False)
    prevlink = subset.iloc[0]['generated_file']
    generated_file = make_entry_for_md(newdate, categ)
    md_filename = f"{newdate}-{categ}.md"
    print('adding to record file', newdate, categ, md_filename, generated_file)
    df = df.append({'date': newdate, 'category': categ, 'md_filename': md_filename, 'generated_file': generated_file},
             ignore_index=True)
    df.to_csv("_data/files_written.csv", index=None)

In [77]:
get_written_files_categ('story')

,date,category,md_filename,generated_file
14,2021-01-12,story,2021-01-12-story.md,_site/categories/story/2021/01/12/story.html
13,2021-01-05,story,2021-01-05-story.md,_site/categories/story/2021/01/05/story.html


In [135]:
# get new data

# HAVE SCRAPED
categ = 'knowledge'
newdata = get_new_data(categ)

newdates, prevdate = write_new_data(categ, newdata)
print("newdates", newdates)
if len(newdates):
    delete_prev_files(prevdate, categ) # not actually deleting rn
    remove_prevdate_from_record(prevdate, categ)

    for date in sorted(newdates):
        print('doing date', date)
        prevlink = add_to_record_get_prevlink(date, categ)
        print("prevlink", prevlink)
        df = read_data_csv_date(date, categ)
        print("len of df", len(df))
        print(df.head())
        write_table_md(df, date, categ, prevlink)

47
all files for categ ['_data/knowledge/1994-05-24-knowledge-100.csv', '_data/knowledge/2019-03-27-knowledge-33.csv', '_data/knowledge/2021-01-05-knowledge-100.csv']
most recent _data/knowledge/2021-01-05-knowledge-100.csv
2021-01-05 knowledge 100
writing new file 2021-01-17
newdates ['2021-01-17']
would here delete 2021-01-05 knowledge
would delete categories/knowledge/_posts/2021-01-05-knowledge.md
would delete _data/knowledge/2021-01-05-knowledge-100.csv
would delete here: date                                                     2021-01-05
category                                                  knowledge
generated_file    _site/categories/knowledge/2021/01/05/knowledg...
Name: 2021-01-05-knowledge.md, dtype: object 15
15
doing date 2021-01-17
adding to record file 2021-01-17 knowledge 2021-01-17-knowledge.md _site/categories/knowledge/2021/01/17/knowledge.html
prevlink None
len of df 47
                                               title  \
0  Political Depolarization of News Ar

In [39]:
def get_all_mdfiles():
    files = []
    for path in Path('categories').rglob('*.md'):
        files.append(str(path))
    return files

In [79]:
from collections import defaultdict

def get_all_csv_files(categs):
    files = defaultdict(list)
    for categ in categs:
        for path in Path(f"_data/{categ}/").rglob('*.csv'):
            if 'written' not in str(path):
                files[categ].append(str(path))
        files[categ] = sorted(files[categ], reverse=False)
    return files

In [80]:
csvfiles = get_all_csv_files(categs)
csvfiles

defaultdict(list,
            {'story': ['_data/story/2021-01-05-story-24.csv'],
             'table2text': ['_data/table2text/2020-07-13-table2text-100.csv',
              '_data/table2text/2021-01-17-table2text-83.csv'],
             'games': ['_data/games/2020-11-15-games-14.csv'],
             'dialogue': ['_data/dialogue/2020-12-29-dialogue-100.csv',
              '_data/dialogue/2020-12-31-dialogue-2.csv'],
             'knowledge': ['_data/knowledge/2020-04-17-knowledge-100.csv',
              '_data/knowledge/2021-01-17-knowledge-83.csv'],
             'poetry': ['_data/poetry/2020-09-25-poetry-14.csv'],
             'image2text': ['_data/image2text/2020-09-28-image2text-100.csv',
              '_data/image2text/2021-01-17-image2text-14.csv']})

In [119]:
# create the written csv record file
written = []
for categ, files in csvfiles.items():
    category = categ
    for i, file in enumerate(files):
        (date, _, _) = parse_filename(file)
        md_filename = f"categories/{categ}/{date}-{categ}.md"
        generated_file = make_entry_for_md(date, category)
        written.append({'date': date, 'category': category, 'md_filename': md_filename, 'generated_file': generated_file, 'data_file': file, 'most_recent': False})

2021-01-05 story 24
2020-07-13 table2text 100
2021-01-17 table2text 83
2020-11-15 games 14
2020-12-29 dialogue 100
2020-12-31 dialogue 2
2020-04-17 knowledge 100
2021-01-17 knowledge 83
2020-09-25 poetry 14
2020-09-28 image2text 100
2021-01-17 image2text 14


In [120]:
written

[{'date': '2021-01-05',
  'category': 'story',
  'md_filename': 'categories/story/2021-01-05-story.md',
  'generated_file': '_site/categories/story/2021/01/05/story.html',
  'data_file': '_data/story/2021-01-05-story-24.csv',
  'most_recent': False},
 {'date': '2020-07-13',
  'category': 'table2text',
  'md_filename': 'categories/table2text/2020-07-13-table2text.md',
  'generated_file': '_site/categories/table2text/2020/07/13/table2text.html',
  'data_file': '_data/table2text/2020-07-13-table2text-100.csv',
  'most_recent': False},
 {'date': '2021-01-17',
  'category': 'table2text',
  'md_filename': 'categories/table2text/2021-01-17-table2text.md',
  'generated_file': '_site/categories/table2text/2021/01/17/table2text.html',
  'data_file': '_data/table2text/2021-01-17-table2text-83.csv',
  'most_recent': False},
 {'date': '2020-11-15',
  'category': 'games',
  'md_filename': 'categories/games/2020-11-15-games.md',
  'generated_file': '_site/categories/games/2020/11/15/games.html',
  'd

In [196]:
def set_most_recent(written_df):
    firsts = written_df.sort_values(by=['category', 'date'], ascending=False).groupby('category').first().reset_index()
    for i, row in firsts.iterrows():
        written_df.loc[(written_df['category'] == row['category']) & (written_df['date'] == row['date']), 'most_recent'] = True
    return written_df

In [122]:
written_df = pd.DataFrame(written)
written_df['prev_link'] = None
written_df = set_most_recent(written_df)

# set the prev links in the file
for categ in categs:
    written_df.loc[written_df['category']==categ].sort_values(by="date", ascending=True, inplace=True)
    shifted = written_df[written_df['category']==categ].shift(1)['generated_file']
    written_df.loc[ written_df['category']==categ, 'prev_link'] = shifted

written_df.to_csv("_data/files_written.csv", index=None)

In [201]:
written_df  # add lengths?

,date,category,md_filename,generated_file,data_file,most_recent,prev_link
0,2021-01-05,story,categories/story/2021-01-05-story.md,_site/categories/story/2021/01/05/story.html,_data/story/2021-01-05-story-24.csv,True,NaN
1,2020-07-13,table2text,categories/table2text/2020-07-13-table2text.md,_site/categories/table2text/2020/07/13/table2t...,_data/table2text/2020-07-13-table2text-100.csv,False,NaN
2,2021-01-17,table2text,categories/table2text/2021-01-17-table2text.md,_site/categories/table2text/2021/01/17/table2t...,_data/table2text/2021-01-17-table2text-83.csv,True,_site/categories/table2text/2020/07/13/table2t...
3,2020-11-15,games,categories/games/2020-11-15-games.md,_site/categories/games/2020/11/15/games.html,_data/games/2020-11-15-games-14.csv,True,NaN
4,2020-12-29,dialogue,categories/dialogue/2020-12-29-dialogue.md,_site/categories/dialogue/2020/12/29/dialogue....,_data/dialogue/2020-12-29-dialogue-100.csv,False,NaN
5,2020-12-31,dialogue,categories/dialogue/2020-12-31-dialogue.md,_site/categories/dialogue/2020/12/31/dialogue....,_data/dialogue/2020-12-31-dialogue-2.csv,True,_site/categories/dialogue/2020/12/29/dialogue....
6,2020-04-17,knowledge,categories/knowledge/2020-04-17-knowledge.md,_site/categories/knowledge/2020/04/17/knowledg...,_data/knowledge/2020-04-17-knowledge-100.csv,False,NaN
7,2021-01-17,knowledge,categories/knowledge/2021-01-17-knowledge.md,_site/categories/knowledge/2021/01/17/knowledg...,_data/knowledge/2021-01-17-knowledge-83.csv,True,_site/categories/knowledge/2020/04/17/knowledg...
8,2020-09-25,poetry,categories/poetry/2020-09-25-poetry.md,_site/categories/poetry/2020/09/25/poetry.html,_data/poetry/2020-09-25-poetry-14.csv,True,NaN
9,2020-09-28,image2text,categories/image2text/2020-09-28-image2text.md,_site/categories/image2text/2020/09/28/image2t...,_data/image2text/2020-09-28-image2text-100.csv,False,NaN


In [195]:
written_df

,date,category,md_filename,generated_file,data_file,most_recent,prev_link
0,2021-01-05,story,categories/story/2021-01-05-story.md,_site/categories/story/2021/01/05/story.html,_data/story/2021-01-05-story-24.csv,True,NaN
1,2020-07-13,table2text,categories/table2text/2020-07-13-table2text.md,_site/categories/table2text/2020/07/13/table2t...,_data/table2text/2020-07-13-table2text-100.csv,False,NaN
2,2021-01-17,table2text,categories/table2text/2021-01-17-table2text.md,_site/categories/table2text/2021/01/17/table2t...,_data/table2text/2021-01-17-table2text-83.csv,True,_site/categories/table2text/2020/07/13/table2t...
3,2020-11-15,games,categories/games/2020-11-15-games.md,_site/categories/games/2020/11/15/games.html,_data/games/2020-11-15-games-14.csv,True,NaN
4,2020-12-29,dialogue,categories/dialogue/2020-12-29-dialogue.md,_site/categories/dialogue/2020/12/29/dialogue....,_data/dialogue/2020-12-29-dialogue-100.csv,False,NaN
5,2020-12-31,dialogue,categories/dialogue/2020-12-31-dialogue.md,_site/categories/dialogue/2020/12/31/dialogue....,_data/dialogue/2020-12-31-dialogue-2.csv,True,_site/categories/dialogue/2020/12/29/dialogue....
6,2020-04-17,knowledge,categories/knowledge/2020-04-17-knowledge.md,_site/categories/knowledge/2020/04/17/knowledg...,_data/knowledge/2020-04-17-knowledge-100.csv,False,NaN
7,2021-01-17,knowledge,categories/knowledge/2021-01-17-knowledge.md,_site/categories/knowledge/2021/01/17/knowledg...,_data/knowledge/2021-01-17-knowledge-83.csv,True,_site/categories/knowledge/2020/04/17/knowledg...
8,2020-09-25,poetry,categories/poetry/2020-09-25-poetry.md,_site/categories/poetry/2020/09/25/poetry.html,_data/poetry/2020-09-25-poetry-14.csv,True,NaN
9,2020-09-28,image2text,categories/image2text/2020-09-28-image2text.md,_site/categories/image2text/2020/09/28/image2t...,_data/image2text/2020-09-28-image2text-100.csv,False,NaN


In [60]:
def get_new_data(categ):
    print(len(arts2[categ]))
    return arts2[categ]

In [10]:
categs = arts2.keys()
categs

dict_keys(['story', 'table2text', 'games', 'dialogue', 'knowledge', 'poetry', 'image2text'])

In [ ]:
for key, values in arts2.items():
     

In [21]:
# trying to fix bad data files

from datetime import datetime
import more_itertools

for key, vals in arts2.items():
    files = Path(f'_data/{key}/').glob("*" + key + "*.csv")
    files = sorted([str(file) for file in files])
    #merge = pd.DataFrame([])
    merge = create_df_from_new_vals(vals)
    print(key)
    for file in files:
        df = pd.read_csv(file)
        print(len(df))
        merge = pd.concat([df, merge], axis=0)
        print(len(merge))
    #print(merge.head())
    merge['pubdate'] = pd.to_datetime(merge['pubdate'], format='%Y-%m-%d', utc=True)
    merge.sort_values(by="pubdate", ascending=True, inplace=True)  # sort by asc in order to write the 100's first
    merge.drop_duplicates(subset="id", inplace=True)
    print(merge.columns)
    write_all_data_files(key, merge, tag="new")
        

story
23
33
24
57
Index(['title', 'pubdate', 'id', 'authors', 'categories', 'search', 'abstract',
       'displaydate'],
      dtype='object')
writing new file 2021-01-05
table2text
100
244
44
288
Index(['title', 'pubdate', 'id', 'authors', 'categories', 'search', 'abstract',
       'displaydate'],
      dtype='object')
writing new file 2020-07-13
writing new file 2021-01-17
games
10
24
Index(['title', 'pubdate', 'id', 'authors', 'categories', 'search', 'abstract',
       'displaydate'],
      dtype='object')
writing new file 2020-11-15
dialogue
30
132
Index(['title', 'pubdate', 'id', 'authors', 'categories', 'search', 'abstract',
       'displaydate'],
      dtype='object')
writing new file 2020-12-29
writing new file 2020-12-31
knowledge
100
247
33
280
100
380
47
427
Index(['title', 'pubdate', 'id', 'authors', 'categories', 'search', 'abstract',
       'displaydate'],
      dtype='object')
writing new file 2020-04-17
writing new file 2021-01-17
poetry
10
24
Index(['title', 'pubdate',

In [206]:
for i, row in written_df.iterrows():
    df = pd.read_csv(row['data_file'])
    prevlink = row['prev_link']
    if type(prevlink) != str:
        prevlink = None
    most_recent = row['most_recent']
    write_table_md(df, row['date'], row['category'], prevlink, most_recent=most_recent)

wrote  categories/story/_posts/2021-01-05-story.md
wrote  categories/table2text/_posts/2020-07-13-table2text.md
wrote  categories/table2text/_posts/2021-01-17-table2text.md
wrote  categories/games/_posts/2020-11-15-games.md
wrote  categories/dialogue/_posts/2020-12-29-dialogue.md
wrote  categories/dialogue/_posts/2020-12-31-dialogue.md
wrote  categories/knowledge/_posts/2020-04-17-knowledge.md
wrote  categories/knowledge/_posts/2021-01-17-knowledge.md
wrote  categories/poetry/_posts/2020-09-25-poetry.md
wrote  categories/image2text/_posts/2020-09-28-image2text.md
wrote  categories/image2text/_posts/2021-01-17-image2text.md


In [113]:
type(None)

NoneType